## An Exploration of Chicago's Array of Things API (via python client)
Array of Things has expanded in Chicago to include more than 100 continuous sensors across the city.

**Resources**
* [Home Page](https://arrayofthings.github.io)
* [python client](https://github.com/UrbanCCD-UChicago/aot-client-py)
* [API documentation](https://arrayofthings.docs.apiary.io)

**TODOs**
  * group measurements by larger cluster than sensor? by project?
  * efficiently query regular interval of data, to annimate
  * normalize measurements, choose reasonable circle radii and color
    * map different sensors to different visualizations

In [ ]:
 # !pip install aot-client

In [ ]:
from aot_client import AotClient

client = AotClient()

# What are the methods/properties of the client?
[_ for _ in dir(client) if not _.startswith('_')]

It appears results will be paginated and we can also pass a filter. From the docs website I see that users may pass a timestamp filter in ISO 8601 format. Let's get measurements from the past 15 minutes.

In [1]:
import datetime

import pandas as pd
from aot_client import AotClient
from aot_client import F


def time_x_mins_ago(minutes:int):
    '''Get formatted time to pass to API filter, relative to current time
    '''
    t = (datetime.datetime.now() - 
         datetime.timedelta(minutes=minutes) + 
         datetime.timedelta(hours=5))  # convert timezone from central to UTC
    t = t.isoformat()
    
    return t[0:19]


def unpack_response(response, page_limit=1000):
    try:
        pages = []
        for i, page in enumerate(response):
            if i + 1 > page_limit:
                break
            pages.extend(page.data)
    except HTTPError as e:
        print(e)    
    finally:
        return pages


def process_observations(obs_df):
    obs_df = obs_df.copy()
    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'], utc=True)
    obs_df['timestamp'] = obs_df['timestamp'].dt.tz_convert('US/Central')
    
    # extract lat/lon to columns
    obs_df['coords'] = obs_df['location'].apply(
        lambda x: x['geometry']['coordinates'])
    obs_df[['lon', 'lat']] = pd.DataFrame(
        obs_df['coords'].tolist(), columns=['lon', 'lat'])
    obs_df = obs_df.drop(columns=['coords'])
    
    # fix positive lon values
    mask = obs_df['lon'] > 0
    if sum(mask) > 0:
        print(f'fixed {sum(mask)} rows with positive lon value')
        obs_df.loc[mask, 'lon'] = obs_df.loc[mask, 'lon'] * -1

    # remove lat/lon values at 0
    mask = (obs_df['lon'] != 0) & (obs_df['lat'] != 0)
    if len(obs_df) - sum(mask) > 0:
        print(f'removed {len(obs_df) - sum(mask)} rows with lat/lon at 0')
        obs_df = obs_df.loc[mask]

    # remove lat values less than 40 degrees
    mask = (obs_df['lat'] > 40)
    if len(obs_df) - sum(mask) > 0:
        print(f'removed {len(obs_df) - sum(mask)} '
              'rows with lat/lon outside Chicago region')
        obs_df = obs_df.loc[mask]
    
    return obs_df

In [ ]:
client = AotClient()

# create filter
f = F('size', '90000')
f &= ('timestamp', 'ge', time_x_mins_ago(5))
# f &= ('sensor', 'image.image_detector.person_total')
# f &= ('time_bucket', 'avg:1 hour')
# f &= ('sensor', 'image.image_detector.car_total')
# f &= ('sensor', 'metsense.tsys01.temperature')

response = client.list_observations(filters=f)
print(response.current_link)
pages = unpack_response(response, page_limit=1)
print(len(pages))
obs_df = pd.DataFrame(pages)
# obs_df = process_observations(obs_df)
print(f"shape: {obs_df.shape}")
obs_df.head()

In [ ]:
len(obs_df['node_vsn'].unique())

In [ ]:
obs_df.groupby('node_vsn')['sensor_path'].nunique()

In [ ]:
import folium
import folium.plugins

def map(df):
    m = folium.Map(location=[df['lat'].mean(), 
                             df['lon'].mean()],
                   tiles='CartoDB dark_matter',
                   zoom_start=10)

    for i, r in df.iterrows():
        folium.CircleMarker(
            location=(r['lat'], r['lon']),              
#             radius=3,
#             color=r['color'],
#             weight=0.5,
            tooltip=f"{r['timestamp']}<br>{r['value']} {r['uom']}",
#             popup=folium.Popup(f"{r['value']} {r['uom']}", max_width=500),
            fill=True
        ).add_to(m)

    folium.plugins.Fullscreen(
        position='topright',
        force_separate_button=True
    ).add_to(m)

    return m

In [ ]:
response.current_link

In [ ]:
len(sensors_df['path'].unique())

In [ ]:
obs_df.head()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

sensors = client.list_sensors()
sensors_df = pd.DataFrame(sensors.data)
sensors_df.head()

@interact_manual
def choose_sensor(sensor=sensors_df['path'].unique()):
    client = AotClient()
    f = F('sensor', sensor)

    response = client.list_observations(filters=f)
    print('API call:', response.current_link)
    pages = unpack_response(response, page_limit=5)
    
    if not pages:
        print('No data found.')
        return None
    
    obs_df = pd.DataFrame(pages)
    obs_df = process_observations(obs_df)
    print(obs_df.shape)
    
    return map(obs_df.drop_duplicates(['lon', 'lat']))

Things I notice:
* multiple temperature, pressure, humidity measures - are they comparable within the same node?
* accelerometers are an interesting choice. I wonder how that data can be used.
* does every node contain the same sensors?

We already stored sensor observations in `obs_df`, but we still don't have context around the data. For that we must pull details about the sensors themselves.

### Interesting Sensors
https://github.com/waggle-sensor/sensors/blob/master/sensors/datasheets/en-d6t.pdf

What's going on with all those **temperature** measures?

In [ ]:
# sort to get most recent data first
df = df.sort_values('timestamp', ascending=False)

# drop duplicates to keep only the most recent value of each measurement
df = df.drop_duplicates(['node_vsn', 'sensor_path'])

temps_df = df.loc[
    (df['parameter'] == 'temperature') & 
    (df['uom'] == 'C')
]
temps_df.groupby('node_vsn')['value'].apply(lambda x: x.unique())

Interesting. These temperatures are quite different from one another, even within the same node.

How about CO levels?

In [ ]:
co_conc_df = df.loc[df['sensor'] == 'co']

co_conc_df

I wonder how there can be negative ppm values. Let's plot it anyway.

In [ ]:
# create a normalized set of values that will be easier scale
co_conc_df['value_norm'] = (
    (co_conc_df['value'] - min(co_conc_df['value'])) / 
    (max(co_conc_df['value']) - min(co_conc_df['value']))
) * 100

In [ ]:
plt.scatter(co_conc_df['lon'], co_conc_df['lat'], s=co_conc_df['value_norm'])
plt.axes().set_aspect('equal')
plt.show()